# ANALISIS DE RIESGOS: Crear dataframe

En este notebook se crea un dataframe que englobe todos los existentes para los diferentes lustros.
Se crea un fichero .csv padre del cual parta los diferentes análisis a realizar

### PREPARACION DE DATASET

In [23]:
import codecs
import pandas as pd
import csv
from os import listdir
from os.path import isfile, join
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

#### Fuente

https://av-info.faa.gov/dd_sublevel.asp?Folder=%5CAID

Todos los datos obtenidos, se realiza mediante el acceso a la pagina web de la fuente. De este modo, en el momento de obtener los datos de estudio, se asegura que estos estén actualizados.

In [52]:
# URL de los datos a analizar: Leyenda
url_legend = 'https://av-info.faa.gov/data/AID/Afilelayout.txt'

# URL de los datos a analizar: Lustros con datos de estudio de 1975 a actualidad
# Obtener ficheros que se desea obtener los datos
lust = [year for year in range(1975, 2025, 5)]
AID_files = [f'a{year}_{str(year+4)[-2:]}.txt' for year in lust]
# El fichero de datos actual, sabemos que es de 2020 a 2025 actualemnte
AID_files[-1:] = 'a2020_25.txt',

with urlopen("https://av-info.faa.gov/dd_sublevel.asp?Folder=%5CAID") as r:
    bs = BeautifulSoup(r.read(), "html.parser")

url_AID = []
for link in bs.find_all("a"):
    # Buscar el texto del fichero en los ficheros creados previamente
    # Se desea aquellos que estan delimitados por \t, por ello buscar /tab/
    if link.next_element in AID_files and '/tab/' in link.get("href"):
        url_AID.append(link.get("href"))

##### Leyenda de datos

Extraer del fichero de leyendas, la referencia a los nombres de columnas para poder validar posteriormente los datos de los datasets.<br>
Se accede al contenido de los datos desde internet, de este modo, existe una referencia actualizada de los datos al obtener estos.

Dar formato al .txt descargado para poder realizar su lectura posterior

In [58]:
# Leemos el contenido de la leyenda
with urlopen(url_legend) as content:
    soup = BeautifulSoup(content, "html.parser")
    soup_lines = str(soup).split('\r\n')

# Transformamos la respuesta en un diccionario con el nombre de la columna y la descripción
legend_df = {"Column_name": [], "Description": []}
# Recorrer las lineas con datos de lineas. Se salta las dos primeras y las tres ultimas
# filas por no tener datos relevantes de la leyenda
for line in soup_lines[2:-3]:
    # Se extrae los 5 primeros caracteres para conocer el nombre de la columna
    legend_df["Column_name"].append(line[0:5].strip())
    # Se extrae la descripcion, esta comienza en la posicion 53
    legend_df["Description"].append(line[53:].strip())
    
# Convertir el diccionario en pandas.Dataframe
legend_df = pd.DataFrame.from_dict(data=legend_df)
legend_df

,Column_name,Description
0,c5,Unique control number used to relate to AID_MA...
1,c1,Type of Event
2,c2,FAR part number
3,c3,Form on which the latest data was received.
4,c4,Agency conducting investigation.
...,...,...
174,c163,2nd Additional cause factor text
175,c183,Supporting Factor Text ...
176,c191,Supporting cause factor B text. O...
177,c229,Date of Birth of PIC


##### Obtener dataframes

Leer y validar todos los ficheros con datos para realizar PHA.<br>

Para obtener los datos, ya que se trata de ficheros muy pesado, se realiza la descarga de los mismos y se trabaja el análisis para crear el dataset desde los ficheros

In [53]:
data_path = 'Datasets/PHA/'
pha_file = 'PHA_Data.csv'

In [54]:
# Descarga de ficheros previamente encontrados con BeautifulSoup
for url, file_name in zip(url_AID, AID_files):
    # Se envía la petición HTTP Get para la obtención de los datos
    data = requests.get(url)

    # Guardamos el archivo de manera local
    with open(data_path+file_name, 'wb')as file:
        file.write(data.content)
        print(f'[+] FILE_CREATED\t{file_name}')

[+] FILE_CREATED	a1975_79.txt
[+] FILE_CREATED	a1980_84.txt
[+] FILE_CREATED	a1985_89.txt
[+] FILE_CREATED	a1990_94.txt
[+] FILE_CREATED	a1995_99.txt
[+] FILE_CREATED	a2000_04.txt
[+] FILE_CREATED	a2005_09.txt
[+] FILE_CREATED	a2010_14.txt
[+] FILE_CREATED	a2015_19.txt
[+] FILE_CREATED	a2020_25.txt


Con los ficheros descargados, se crea un dataframe que albergue todos los datos. Para ello, se validará que las columnas existentes en los sucesivos ficheros, existen en la leyenda previamente extraida.

In [79]:
def validate_df(df) -> None:
    """
    Validar las columnas existentes en el dataframe creado
    """
    for column in df.columns:
        # Saltar la columna "end_of_record"
        if column == 'end_of_record':
            continue
        # Acceder al registro de la leyenda y comprobar que hay resultados
        if legend_df.loc[legend_df['Column_name'] == column].empty:
            raise AssertionError(column)

In [81]:
# Ruta de acceso a los ficheros descargados
datasets = [data_path + file for file in AID_files]

# Crear un dataframe que posteriormente ira albergando el resto de dataframes que se crean
try:
    # Crear un dataframe que posteriormente ira albergando el resto de dataframes que se crean
    dataset = datasets[0]
    doc = codecs.open(dataset, 'rU')
    df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
    
    # Validar dataset
    validate_df(df)
        
    print('[+] Success: ', dataset)

    for dataset in datasets[1:]:
        doc = codecs.open(dataset, 'rU')
        aux_df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)

        # Validar dataset
        validate_df(aux_df)            

        # Añadir nuevos datos al dataframe
        # Se añade los nuevos registros ordenados por columnas
        df = pd.concat([df, aux_df],axis=0)
        del aux_df
        print('[+] Success: ', dataset)
        
except AssertionError as e:
    print(f'[-] BAD_FILE\t{dataset}\tCOLUMN_ERROR\t{e.__str__()}')
    del df
    
    doc = codecs.open(dataset, 'rU')
    error_df = pd.read_csv(doc, sep='\t', on_bad_lines='skip', low_memory=False)
    display(error_df)

[+] Success:  Datasets/PHA/a1975_79.txt
[+] Success:  Datasets/PHA/a1980_84.txt
[+] Success:  Datasets/PHA/a1985_89.txt
[+] Success:  Datasets/PHA/a1990_94.txt
[+] Success:  Datasets/PHA/a1995_99.txt
[+] Success:  Datasets/PHA/a2000_04.txt
[+] Success:  Datasets/PHA/a2005_09.txt
[+] Success:  Datasets/PHA/a2010_14.txt
[+] Success:  Datasets/PHA/a2015_19.txt
[+] Success:  Datasets/PHA/a2020_25.txt


###### Nota:
Durante la validación, el fichero "a1990_94.txt" tiene un error en la columna "32":
[-] BAD_FILE	Datasets/PHA/a1990_94.txt	COLUMN_ERROR	32

Tras realizar las comprobaciones pertinentes, se observa que se debe a un mal tipado en la fuente de los datos y esta columna debería ser "c32". Por lo tanto, se modifica el fichero y se vuelve a realizar las comprobaciones

Una vez validado el dataframe, debido al gran volumen de datos que existe, se guarda el fichero en formato .parquet

In [82]:
df.to_parquet(data_path + pha_file, index = False)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.